# Desafio para área de Data Engineering
---

## Case
Ingestão de novas informações
Você foi contratado pela DHAUZ como engenheiro de dados para trabalhar na operação de um
ambiente analítico de dados e recebe como tarefa realizar a análise e manipulação de informações
objetivando responder a algumas perguntas de negócio.

O seu desafio é realizar a leitura dessas informações de uma fonte externa, aplicar os processamentos
necessários e então responder as questões levantadas.

### Tabelas
<img src="utils/tabela.png">

### Criação do Repositório
 - Git init
 - Criação de um venv para o projeto
 - Setar o kernel do venv para o jupyter
 - Instalar dependencias
 - Criar arquivo .env
 - Configurar .gitignore


In [ ]:
#Criação de um ambiente virtual para isolamento das dependencias do projeto, selecione o novo kernel
!python -m venv venv

: 

In [2]:
#Instale para conseguir se comunicar com o Jupyter
!pip install ipykernel

In [11]:
#Instale para se connectar a instancia do bd, pyenv para carregar arquivo .env
#!pip install mysql-connector-python
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


In [2]:
#Criando função de conexão e consulta
import mysql.connector  
import os, queries
from dotenv import load_dotenv; load_dotenv() #carrega segredos no .env

def make_connection():
    """Cria conexão com o banco de dados através do arquivo .env de configuração

    Returns:
        _type_: _mysql.connector.connection_cext.CMySQLConnection_
    """    
    con = mysql.connector.connect(user=os.getenv('USER'),password=os.getenv('PSWD'), \
        host=os.getenv('HOST'), database=os.getenv('DB'))
    return con

def select(query):
    """_Realiza o processo de executar consulta_

    Args:
        query (_type_): _string_

    Returns:
        _type_: _list_
    """    
    con = make_connection()
    cursor = con.cursor()
    cursor.execute(query)
    result = cursor.fetchall()

    if con.is_connected():
        cursor.close()
        con.close()
        
    return result

#1
### Questões

1. Qual é o endereço (carteira) com maior volume de transações enviadas?
---
> Explicação: 
Nessa primeira questão utilizo duas CTE que me ajudam a agregar dados, tratar datas (também usadas posteriormente)
e com a soma da quantidade de transações agrupada pela carteira que fez o envio, é possível identificar a que teve um volume maior.
       <font size="1">
```sql 
WITH 
ntr_transactions AS (
-- Note: 0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday, 4 = Friday, 5 = Saturday, 6 = Sunday.
SELECT SentDate, MONTH(SentDate) as _month,  DAY(SentDate) AS month_day, WEEKDAY(SentDate) week_day, 
       AddressOrigin AS address_origin, 1 as n_t 
FROM raw_transactions_table rtt 
WHERE status in ('Confirmed')
),

ntr_agg_transactions AS (
SELECT _month, month_day, week_day, address_origin, sum(n_t) as n_transactions
FROM ntr_transactions 
WHERE _month = 1
group by 1,2,3,4
)

-- CARTEIRA COM MAIOR TRANSACACOES
SELECT address_origin, SUM(n_transactions) as sum_n_transactions
FROM ntr_agg_transactions GROUP BY 1 ORDER BY 2 DESC LIMIT 1```
</font>

In [4]:
print(select(queries.QUESTAO_1))
    

[('A-99', Decimal('89'))]


2. Qual é o dia do mês com maior volume de transações realizadas?
> Explicação: 
Utilizando as mesmas duas CTE's da primeira questão, agregando a coluna de dias criada é possível somar as transacoes
e obter o dia com a maior quantidade.
<font size="1">
``` sql
WITH 
ntr_transactions AS (
-- Note: 0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday, 4 = Friday, 5 = Saturday, 6 = Sunday.
SELECT SentDate, MONTH(SentDate) as _month,  DAY(SentDate) AS month_day, WEEKDAY(SentDate) week_day, 
       AddressOrigin AS address_origin, 1 as n_t 
FROM raw_transactions_table rtt 
WHERE status in ('Confirmed')
),

ntr_agg_transactions AS (
SELECT _month, month_day, week_day, address_origin, sum(n_t) as n_transactions
FROM ntr_transactions 
WHERE _month = 1
group by 1,2,3,4
)

-- DIA COM MAIOR TRANSACOES
SELECT month_day, SUM(n_transactions) as sum_n_transactions
FROM ntr_agg_transactions GROUP BY 1 ORDER BY 2 DESC LIMIT 1
``` 
</font>

In [2]:
print(select(queries.QUESTAO_2))

[(7, Decimal('286'))]


3. Em qual dia da semana geralmente mais transações são realizadas?
> Explicação: Para essa questão além das duas cte's utilizadas primeiramente, adicionei mais CTE
que tem a função de abrir os dias das semanas dentro dos 31 dias do mês para calcular média com frequencia correta
<font size="1">
```sql
WITH 
ntr_transactions AS (
-- Note: 0 = Monday, 1 = Tuesday, 2 = Wednesday, 3 = Thursday, 4 = Friday, 5 = Saturday, 6 = Sunday.
SELECT SentDate, MONTH(SentDate) as _month,  DAY(SentDate) AS month_day, WEEKDAY(SentDate) week_day, 
       AddressOrigin AS address_origin, 1 as n_t 
FROM raw_transactions_table rtt 
WHERE status in ('Confirmed')
),

ntr_agg_transactions AS (
SELECT _month, month_day, week_day, address_origin, sum(n_t) as n_transactions
FROM ntr_transactions 
WHERE _month = 1
group by 1,2,3,4
),
-- DIA SEMANA AVG TRANS REALIZADA
ntr_agg AS (
SELECT month_day, week_day, SUM(n_transactions) as nn_transactions
FROM ntr_agg_transactions
GROUP BY 1,2 ORDER BY 1) 

SELECT week_day, AVG(nn_transactions)
FROM ntr_agg 
GROUP BY 1 ORDER BY 2 DESC LIMIT 1
``` 
</font>

In [3]:
print(select(queries.QUESTAO_3))

[(2, Decimal('221.5000'))]


4. Quais transações possuem condições atípicas e precisam ser validadas com o time responsável pela disponibilização dos dados?
> Explicação: Dois pontos, alguns campos de retorno status pelo que foi solicitado não se dá pra saber se deve ou não serem computados.
Atualização: identificando outros pontos para as últimas questões percebi que ao manipular o ID que a importação de dados do mês vieram dados repetidos do mês um, estes serão removidos para todas as consultas
* Resposta: Todas as transações que tem status diferente de confirmed, transações duplicadas na importação do mês um na importação do mês dois

<font size="1">
```sql
--Esta consulta retorna uma matriz de dados mês, status, qtd, os valores de Janeiro / Confirmed são os considerados, o resto atípico
SELECT month(SentDate) AS _month, status, count(1)
FROM raw_transactions_table rtt 
GROUP BY 1,2
``` 
</font>

In [ ]:
print(select(queries.QUESTAO_4))

5. Qual a carteira com o maior saldo final? (considere que todas as carteiras estejam zeradas no início das análises e que seja possível existir saldo negativo).
> Explicação: CTE's criadas para coletar o que foi enviado e o que foi recebido, após é possível gerar um valor de saldo 
e chegar até o balanço mais positivo
<font size="1">
```sql
WITH 
ntr_rtt_clean AS(
	SELECT DISTINCT IdTransaction as id, AddressOrigin as origin, AddressDestination AS destination, 
	TotalSent, MONTH(SentDate) as _month,
	-- Correção do campo TotalSent retiro da , e cast
	CAST(REPLACE(TotalSent,',','') AS FLOAT) as amount,
	SentDate 
	FROM raw_transactions_table rtt 
	WHERE Status = 'Confirmed'
	ORDER BY id
),
ntr_amount_origin AS (
	SELECT origin, SUM(amount) AS amount
	FROM ntr_rtt_clean
	GROUP BY 1
),

ntr_amount_destination AS (
	SELECT destination, SUM(amount) AS amount
	FROM ntr_rtt_clean
	GROUP BY 1
),

ntr_amount_balance AS (
	SELECT nao.origin as wallet, (nad.amount - nao.amount) as balance
	FROM ntr_amount_origin nao 
	LEFT JOIN ntr_amount_destination nad
	ON nao.origin = nad.destination
)
	
SELECT wallet, balance FROM ntr_amount_balance ORDER BY balance DESC LIMIT 1
```
</font>

In [2]:
print(select(queries.QUESTAO_5))

[('A-30', 12715115.0)]


6. Questões 1-4 pt 2:
    * 6.1. Considerando que a carteira origem é responsável por pagar as taxas de envio, qual carteira
    seria responsável pelo maior pagamento de taxas em janeiro de 2021?
    * 6.2. E em fevereiro de 2021?
    * 6.3. Qual é o id da transação com a maior taxa paga?
    * 6.4. Qual é a média de taxa paga considerando todas as transações realizadas?
> Explicação: Após entender alguns problemas que a base tinha como duplicidade, necessidade de corrigir o TotalSent, montei um CTE que serve como base para resolver todas as últimas questões e estruturei elas em formato de um relatório
<font size="1">
```sql
WITH 
-- Garante uma base analítica correta para as resoluções
ntr_rtt_clean AS(
	SELECT DISTINCT IdTransaction as id, AddressOrigin as address, 
	TotalSent, MONTH(SentDate) as _month,
	-- Correção do campo TotalSent retiro da , e cast
	CAST(REPLACE(TotalSent,',','') AS FLOAT) as amount,
	SentDate 
	FROM raw_transactions_table rtt 
	WHERE Status = 'Confirmed'
	ORDER BY id
),

-- Agrupa os valores de taxas necessários para as resoluções
ntr_amount_fee AS (
 SELECT ntc.id, ntc.address, ntc.amount, rtf.`fee-percentage`, 
 		ntc.amount * (rtf.`fee-percentage`/100) as tax_amount, ntc._month, 
 	    CONCAT(ntc.id, ntc._month) as id_month
 FROM ntr_rtt_clean ntc 
 LEFT JOIN raw_transactions_fee rtf 
 ON ntc.amount >= rtf.`range-start`  
 AND ntc.amount <= rtf.`range-end` 
 ORDER BY id
) 


-- 4
SELECT 'q4' as questao, 
       'media' as v1, 
       TRUNCATE(AVG(tax_amount),2) as v2
FROM ntr_amount_fee

UNION ALL

-- 3
(SELECT 'q3' as questao,
	id as v1, 
	tax_amount as v2
FROM ntr_amount_fee
ORDER BY tax_amount DESC LIMIT 1)

UNION ALL

-- 2
(
SELECT 'q2' as questao, 
	address as v1, 
	TRUNCATE(SUM(tax_amount),2) as v2
FROM ntr_amount_fee
WHERE _month = 2
GROUP BY 1 ORDER BY 2 DESC LIMIT 1 
)

UNION ALL
-- 1
(
SELECT 'q1' as questao,
	address as v1, 
	TRUNCATE(SUM(tax_amount),2) as v2
FROM ntr_amount_fee
WHERE _month = 1
GROUP BY 1 ORDER BY 2 DESC LIMIT 1
)
``` 
</font>

In [3]:
print(select(queries.QUESTAO_6))

[('q4', 'media', 21711.24), ('q3', 'ID635', 33485.05), ('q2', 'A-87', 3512161.19), ('q1', 'A-43', 64813138.38)]


### Script
> Por fim será criado o script result.py com os resultados de todas as questões

In [21]:
#Criando função de conexão e consulta
import mysql.connector  
import os, queries
from dotenv import load_dotenv; load_dotenv() #carrega segredos no .env

def make_connection():
    """Cria conexão com o banco de dados através do arquivo .env de configuração

    Returns:
        _type_: _mysql.connector.connection_cext.CMySQLConnection_
    """    
    con = mysql.connector.connect(user=os.getenv('USER'),password=os.getenv('PSWD'), \
        host=os.getenv('HOST'), database=os.getenv('DB'))
    return con

def select(query):
    """_Realiza o processo de executar consulta_

    Args:
        query (_type_): _string_

    Returns:
        _type_: _list_
    """    
    con = make_connection()
    cursor = con.cursor()
    cursor.execute(query)
    result = cursor.fetchall()

    if con.is_connected():
        cursor.close()
        con.close()
        
    return result

def pretty_print(result_list, question_number):
    if question_number < 6:
        print(f'q{question_number}\t{result_list[0][0]}\t{result_list[0][1]}')
    else:
        for q in result_list:
            print(f'{q[0]}\t{q[1]}\t{q[2]}')

In [24]:
print('Questao\tV1\tV2')
pretty_print(select(queries.QUESTAO_1),1)
pretty_print(select(queries.QUESTAO_2),2)
pretty_print(select(queries.QUESTAO_3),3)
pretty_print(select(queries.QUESTAO_4),4)
pretty_print(select(queries.QUESTAO_5),5)
print('-'*20)
pretty_print(select(queries.QUESTAO_6),6)

Questao	V1	V2
q1	A-99	45
q2	7	139
q3	3	106.0000
q4	1	Confirmed
q5	A-30	12715115.0
--------------------
q4	media	21711.24
q3	ID635	33485.05
q2	A-87	3512161.19
q1	A-43	64813138.38


In [25]:
# freeze de dependencias para requiriments
!pip freeze > requirements.txt